In [ ]:
!pip install beautifulsoup4 requests openpyxl


In [ ]:
from bs4 import BeautifulSoup as bs
from google.colab import drive
import os
import re
import requests as rq
import json
import time
from openpyxl import Workbook
from typing import Optional, Union, Dict, List
import random

In [ ]:

import pandas as pd

from google.colab import drive

drive.mount('/content/drive', force_remount=True)

In [ ]:
cd /content/drive/MyDrive/Eudops_Project

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-11-1234376beb48>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('cd', '/content/drive/MyDrive/Eudops_Project')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPytho

This is the code for getting all the reviews from Saladlab

In [ ]:
class Coupang:
    headers = {
        "authority": "weblog.coupang.com",
        "scheme": "https",
        "origin": "https://www.coupang.com",
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "macOS",
        "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Whale/3.20.182.14 Safari/537.36",
        "cookie": "PCID=31489593180081104183684; _fbp=fb.1.1644931520418.1544640325; gd1=Y; X-CP-PT-locale=ko_KR; MARKETID=31489593180081104183684; sid=03ae1c0ed61946c19e760cf1a3d9317d808aca8b; overrideAbTestGroup=%5B%5D; x-coupang-origin-region=KOREA; x-coupang-accept-language=ko_KR;"
    }

    @staticmethod
    def get_product_code(url: str) -> str:
        """Extract PRODUCT CODE from the provided URL."""
        return url.split('products/')[-1].split('?')[0]

    def fetch_reviews(self, product_url: str, pages: int) -> List[List[Dict[str, Union[str, int]]]]:
        product_code = self.get_product_code(product_url)

        #URL only containing the reviews, from the latest
        urls = [
            # f'https://www.coupang.com/vp/product/reviews?productId={product_code}&page={page}&size=5&sortBy=DATE_DESC&ratings=&q=&viRoleCode=3&ratingSummary=true'
            f'https://www.coupang.com/vp/product/reviews?productId={product_code}&page={page}&size=5&sortBy=ORDER_SCORE_ASC&ratings=&q=&viRoleCode=3&ratingSummary=true'
            for page in range(1, pages + 1)
        ]
        self.headers['referer'] = product_url

        print(urls[0])

        results = []
        count = 0
        with rq.Session() as session:
            for url in urls:
                count += 1
                print(f'Page num {count} collection started...')

                response = session.get(url, headers=self.headers)
                results.extend(self.parse_reviews(response.text))
                time.sleep(random.randint(1,3))  # Be polite with the server


                print(f'Page num {count} collected')

        return results

    def parse_reviews(self, html: str) -> List[Dict[str, Union[str, int]]]:
        soup = bs(html, 'html.parser')
        articles = soup.select('article.sdp-review__article__list')
        review_data = []


        for article in articles:
            #Review UID
            review_id = article.select_one('button.sdp-review__article__list__help__report-btn')['data-review-id']

            # Username
            user_name = article.select_one('span.sdp-review__article__list__info__user__name')

            # User UID
            user_uid = user_name['data-member-id']

            if user_name == None or user_name.text == '':
                user_name = '-'
            else:
                user_name = user_name.text.strip()



            # Rating
            rating = article.select_one('div.sdp-review__article__list__info__product-info__star-orange')
            if rating == None:
                rating = 0
            else :
                rating = int(rating.attrs['data-rating'])

            # Product Name
            prod_name = article.select_one('div.sdp-review__article__list__info__product-info__name')
            if prod_name == None or prod_name.text == '':
                prod_name = 'N/A'
            else:
                prod_name = prod_name.text.strip()

            # Date
            date = article.select_one('div.sdp-review__article__list__info__product-info__reg-date').text.strip()

            # Seller
            seller = article.select_one('div.sdp-review__article__list__info__product-info__seller_name')
            if seller == None or seller.text == '':
                seller = 'N/A'
            else:
                seller = seller.text.strip()[5:]

            # Review Title/Headline
            headline = article.select_one('div.sdp-review__article__list__headline')
            if headline == None or headline.text == '':
                headline = 'N/A'
            else:
                headline = headline.text.strip()

            # Review Content
            review_content = article.select_one('div.sdp-review__article__list__review > div')
            if review_content == None :
                review_content = 'N/A'
            else:
                review_content = re.sub('[\n\t]','',review_content.text.strip())

            # Survey Feedback
            answer = article.select_one('span.sdp-review__article__list__survey__row__answer')
            if answer == None or answer.text == '':
                answer = 'N/A'
            else:
                answer = answer.text.strip()


            review_details = {
                'review_uid': review_id,
                'username_2': user_name,
                'user_uid': user_uid,
                'rating': rating,
                'product_name': prod_name,
                'review_date': date,
                'seller_name': seller,
                'review_title': headline,
                'review_content': review_content,
                'survey_answer': answer
            }

            review_data.append(review_details)


        return review_data

In [ ]:
# Replace product URL and number of pages to scrape
product_url = "https://www.coupang.com/vp/products/1650396516?itemId=2811829221&vendorItemId=75168794077&sourceType=CATEGORY&categoryId=176498&isAddedCart="
pages = 45  # Total number of reviews / 5

# Create an instance of the Coupang class
coupang_scraper = Coupang()
reviews = coupang_scraper.fetch_reviews(product_url, pages)
df_reviews = pd.DataFrame(reviews)

print(df_reviews.sample(5))

In [ ]:
df_reviews.to_csv('/content/drive/My Drive/Eudops_Project/Coupang_Collected_Reviews/에르게니아.csv', mode='a', index=False, header=True)